In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick

## Data Parsing

In [ ]:
def parse_data(file_path):
    df = pd.read_csv(file_path)
    if all(elem in df.columns for elem in ["Date", "High", "Low", "Open", "Close"]):
        df = df[["Date", "High", "Low", "Open", "Close"]]
    else:
        raise Exception("Required columns: {}".format(["Date", "High", "Low", "Open", "Close"]))
        return None
    return df

In [ ]:
BTC = parse_data("../input/cryptocurrencypricehistory/coin_Bitcoin.csv")
BTC.head()

## Visualization

In [ ]:
def plot_ohlc(data):
    fig1, ax1 = plt.subplots(figsize=(12, 3), constrained_layout=True)
    
    for i in data.index:
        ax1.vlines(x=i, ymin=data["Low"][i], ymax=data["High"][i], color="black", linewidth=1)
        if data["Close"][i] > data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i], color="green", linewidth=3)
        elif data["Close"][i] < data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Close"][i], ymax=data["Open"][i], color="red", linewidth=3)
        elif data["Close"][i] == data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i]+0.000001, color="black", linewidth=6)
        
    ax1.grid()

plot_ohlc(BTC.tail(100))

## Back Testing

In [ ]:
def buy_signal_each(x):
    s1, s2, s3, s4 = [8, 5, 3, 1]
    h = s1 + s2 + s3 + s4
    c1 = np.all(x[h-s1:h] < np.min(x[h-(s1+s2):h-s1]))
    c2 = np.all(x[h-(s1+s2):h-s1] < np.min(x[h-(s1+s2+s3):h-(s1+s2)]))
    c3 = np.all(x[h-(s1+s2+s3):h-(s1+s2)] < np.min(x[h-(s1+s2+s3+s4):h-(s1+s2+s3)]))
    return c1 and c2 and c3

def buy_signal_one(x):
    s1, s2, s3, s4 = [8, 5, 3, 1]
    h = s1 + s2 + s3 + s4 - 1
    c1 = x[h] < x[h-(s1)]
    c2 = x[h] < x[h-(s1+s2)]
    c3 = x[h] < x[h-(s1+s2+s3)]
    return c1 and c2 and c3

def buy_signal_two(x):
    s1, s2, s3, s4 = [8, 5, 3, 1]
    h = s1 + s2 + s3 + s4 - 1
    c1 = x[h] < x[h-(s1)]
    c2 = x[h-(s1)] < x[h-(s1+s2)]
    c3 = x[h-(s1+s2)] < x[h-(s1+s2+s3)]
    return c1 and c2 and c3

def sell_signal_each(x):
    s1, s2, s3, s4 = [8, 5, 3, 1]
    h = s1 + s2 + s3 + s4
    c1 = np.all(x[h-s1:h] > np.min(x[h-(s1+s2):h-s1]))
    c2 = np.all(x[h-(s1+s2):h-s1] > np.min(x[h-(s1+s2+s3):h-(s1+s2)]))
    c3 = np.all(x[h-(s1+s2+s3):h-(s1+s2)] > np.min(x[h-(s1+s2+s3+s4):h-(s1+s2+s3)]))
    return c1 and c2 and c3

def sell_signal_one(x):
    s1, s2, s3, s4 = [8, 5, 3, 1]
    h = s1 + s2 + s3 + s4 - 1
    c1 = x[h] > x[h-(s1)]
    c2 = x[h] > x[h-(s1+s2)]
    c3 = x[h] > x[h-(s1+s2+s3)]
    return c1 and c2 and c3

def sell_signal_two(x):
    s1, s2, s3, s4 = [8, 5, 3, 1]
    h = s1 + s2 + s3 + s4 - 1
    c1 = x[h] > x[h-(s1)]
    c2 = x[h-(s1)] > x[h-(s1+s2)]
    c3 = x[h-(s1+s2)] > x[h-(s1+s2+s3)]
    return c1 and c2 and c3

In [ ]:
def generate_fibo_signals(data, t=17):
    signals = [None] * (t-1)
    close_prices = data["Close"].to_numpy()
    for i in range(t, len(close_prices)+1):
        if buy_signal_one(close_prices[i-t:i]):
            signals.append("buy")
        elif sell_signal_one(close_prices[i-t:i]):
            signals.append("sell")
        else:
            signals.append(None)
    return np.array(signals)

In [ ]:
BTC["fibo_timing"] = generate_fibo_signals(BTC)
BTC

In [ ]:
def test_trading_strategy(data, holding_period=2):
    t = 17
    holding_period = holding_period
    returns = [1] * (t-1)
    position = False
    n_positions = 0
    remaining_days = 0
    entry_price = None
    exit_price = None
    close_prices = data["Close"].to_numpy()

    for i in range(t, len(close_prices) - holding_period):
        # open position
        if not position:
            # buy signal
            if buy_signal_one(close_prices[i-t:i]):
                position = "buy"
                n_positions += 1
                remaining_days = holding_period
                entry_price = close_prices[i-1]
            # sell signal
            elif sell_signal_one(close_prices[i-t:i]):
                position = "sell"
                n_positions += 1
                remaining_days = holding_period
                entry_price = close_prices[i-1]
            returns.append(1)
        # close position
        elif remaining_days == 0:
            exit_price = close_prices[i-1]
            if position == "buy":
                r = exit_price / entry_price
            elif position == "sell":
                r = entry_price / exit_price
            position = False
            returns.append(r)
        # reduce remaining days
        else:
            remaining_days -= 1
            returns.append(1)
    returns += [1] * holding_period
        
    # Return Statistics
    returns = np.array(returns)
    equity = np.cumprod(returns)
    quality = np.sum(returns > 1) / n_positions

    fig, ax = plt.subplots(figsize=(12, 6), constrained_layout=True)

    ax.plot(equity, color="blue")
    ax.set_title("Equity Return / Signal quality: {:.2%}".format(quality))
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0, decimals=2))
    ax.grid()
        
    return returns, equity, quality

BTC_stat = test_trading_strategy(BTC, holding_period=3)

In [ ]:
def plot_ohlc_with_indicator(data, t=17):
    factor = 2500 / len(data)
    
    fig1, ax1 = plt.subplots(figsize=(12, 6), constrained_layout=True)
    
    for i in data.index:
        ax1.vlines(x=i, ymin=data["Low"][i], ymax=data["High"][i], color="black", lw=1)
        if data["Close"][i] > data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i], color="green", lw=3)
        elif data["Close"][i] < data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Close"][i], ymax=data["Open"][i], color="red", lw=3)
        elif data["Close"][i] == data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i]+0.000001, color="black", lw=6)
    
    for i in data.index:
        if data["fibo_timing"][i] == "buy":
            ax1.scatter(i, data["Low"][i] * 0.99, marker="^", s=factor*2, c="green")
        elif data["fibo_timing"][i] == "sell":
            ax1.scatter(i, data["High"][i] * 1.01, marker="v", s=factor*2, c="red")
    ax1.grid()

plot_ohlc_with_indicator(BTC[1750:2000])

In [ ]:
def EMA(x, data):
    ema_arr = [0] * (x-1)
    k = 2 / (x + 1)
    for i in range(x, len(data)+1):
        if i == x:
            ema_arr.append(np.mean(data[i-x:i]))
        else:
            ema_arr.append(data[i-1] * k + ema_arr[i-2] * (1-k))
    return np.array(ema_arr)

def fibo_EMA(data):
    fibo = [2, 3, 5, 8, 13, 21, 34, 55]
    dic = dict()
    for n in fibo:
        dic["ema" + str(n)] = EMA(n, data)
    fibo_ema = pd.DataFrame(dic).to_numpy().mean(axis=1)
    return fibo_ema

In [ ]:
BTC["fibo_EMA"] = fibo_EMA(BTC["Close"].to_numpy())
BTC.head()

In [ ]:
def buy_signal_ema(close, ema, threshold):
    if close < ema or abs(close - ema) < threshold:
        return True
    else:
        return False

def sell_signal_ema(close, ema, threshold):
    if close > ema or abs(ema - close) < threshold:
        return True
    else:
        return False

In [ ]:
def generate_fibo_strategy_signals(data, t=17):
    threshold = 10
    signals = [None] * (t-1)
    close_prices = data["Close"].to_numpy()
    ema_arr = data["fibo_EMA"].to_numpy()
    for i in range(t, len(close_prices)+1):
        if buy_signal_one(close_prices[i-t:i]) and buy_signal_ema(close_prices[i-1], ema_arr[i-1], threshold):
            signals.append("buy")
        elif sell_signal_one(close_prices[i-t:i]) and sell_signal_ema(close_prices[i-1], ema_arr[i-1], threshold):
            signals.append("sell")
        else:
            signals.append(None)
    return np.array(signals)

In [ ]:
BTC["fibo_strategy"] = generate_fibo_strategy_signals(BTC)
BTC

In [ ]:
def test_trading_strategy(data, holding_period=2):
    t = 17
    threshold = 10
    holding_period = holding_period
    returns = [1] * (t-1)
    position = False
    n_positions = 0
    remaining_days = 0
    entry_price = None
    exit_price = None
    ema_arr = data["fibo_EMA"].to_numpy()
    close_prices = data["Close"].to_numpy()

    for i in range(t, len(close_prices) - holding_period):
        # open position
        if not position:
            # buy signal
            if buy_signal_one(close_prices[i-t:i]) and buy_signal_ema(close_prices[i-1], ema_arr[i-1], threshold):
                position = "buy"
                n_positions += 1
                remaining_days = holding_period
                entry_price = close_prices[i-1]
            # sell signal
            elif sell_signal_one(close_prices[i-t:i]) and sell_signal_ema(close_prices[i-1], ema_arr[i-1], threshold):
                position = "sell"
                n_positions += 1
                remaining_days = holding_period
                entry_price = close_prices[i-1]
            returns.append(1)
        # close position
        elif remaining_days == 0:
            exit_price = close_prices[i-1]
            if position == "buy":
                r = exit_price / entry_price
            elif position == "sell":
                r = entry_price / exit_price
            position = False
            returns.append(r)
        # reduce remaining days
        else:
            remaining_days -= 1
            returns.append(1)
    returns += [1] * holding_period
        
    # Return Statistics
    returns = np.array(returns)
    equity = np.cumprod(returns)
    quality = np.sum(returns > 1) / n_positions

    fig, ax = plt.subplots(figsize=(12, 6), constrained_layout=True)

    ax.plot(equity, color="blue")
    ax.set_title("Equity Return / Signal quality: {:.2%}".format(quality))
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0, decimals=2))
    ax.grid()
        
    return returns, equity, quality

BTC_stat = test_trading_strategy(BTC, holding_period=3)

In [ ]:
def plot_ohlc_with_strategy(data, t=17):
    factor = 2500 / len(data)
    
    fig1, ax1 = plt.subplots(figsize=(12, 6), constrained_layout=True)
    
    ax1.plot(data.index, data["fibo_EMA"], c="orange")
    for i in data.index:
        ax1.vlines(x=i, ymin=data["Low"][i], ymax=data["High"][i], color="black", lw=1)
        if data["Close"][i] > data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i], color="green", lw=3)
        elif data["Close"][i] < data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Close"][i], ymax=data["Open"][i], color="red", lw=3)
        elif data["Close"][i] == data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i]+0.000001, color="black", lw=6)
    
    for i in data.index:
        if data["fibo_timing"][i] == "buy":
            ax1.scatter(i, data["Close"][i] * 0.99, marker="^", s=factor, c="green")
        elif data["fibo_timing"][i] == "sell":
            ax1.scatter(i, data["Close"][i] * 1.01, marker="v", s=factor, c="red")
    
    for i in data.index:
        if data["fibo_strategy"][i] == "buy":
            ax1.scatter(i, data["Close"][i] * 0.95, marker="o", s=factor*2, c="green")
        elif data["fibo_strategy"][i] == "sell":
            ax1.scatter(i, data["Close"][i] * 1.05, marker="o", s=factor*2, c="red")
    ax1.grid()

plot_ohlc_with_strategy(BTC[1900:2000])